In [1]:
import csv
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import os
import re
import unicodedata
from datetime import datetime


In [2]:
# 1. Definir la URL base y las rutas de los archivos de entrada y salida
url_base = "https://www.inmuebles24.com/departamentos-en-renta-en-"
csv_colonias_filename = "/Users/renataramirez/Documents/mlops-repo/data/raw/CP/CPdescarga - Distrito_Federal.csv"

In [3]:
# Obtener solo la fecha actual para crear una carpeta de versionado
timestamp = datetime.now().strftime("%Y-%m-%d")
output_directory = f"/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/por_colonia_{timestamp}"

In [4]:
# 2. Crear listas para almacenar los datos
colonias_sin_resultados = []

## Funciones

In [5]:
def normalizar_nombre(nombre):
    """
    Normaliza el nombre (de municipio o colonia) para ser usado en la URL y en el nombre del archivo.
    - Convierte a minúsculas.
    - Elimina acentos y caracteres especiales.
    - Reemplaza espacios por guiones.
    """
    # Convertir a minúsculas
    nombre = nombre.lower()
    # Eliminar acentos
    nombre = unicodedata.normalize('NFKD', nombre).encode('ascii', 'ignore').decode('utf-8')
    # Reemplazar espacios y caracteres no alfanuméricos por guiones
    nombre = re.sub(r'[\s\W]+', '-', nombre)
    return nombre

In [6]:
def leer_colonias_csv(filepath):
    """
    Lee el archivo CSV y extrae los nombres únicos de las colonias.
    """
    colonias = []
    try:
        with open(filepath, 'r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if 'd_asenta' in row and row['d_asenta']:
                    colonias.append(row['d_asenta'].strip())
    except FileNotFoundError:
        print(f"Error: El archivo '{filepath}' no se encontró.")
        return None
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo CSV: {e}")
        return None
    
    return sorted(list(set(colonias)))

In [ ]:
def scrape_current_page(driver, nombre_colonia):
    """
    Extrae los datos de todas las propiedades en la página actual.
    """
    inmuebles_en_pagina = []
    
    try:
        listings = driver.find_elements(By.CSS_SELECTOR, '.postingCardLayout-module__posting-card-layout')
        
        for listing in listings:
            inmueble = {}
            try:
                title_container = listing.find_element(By.CSS_SELECTOR, '.postingCard-module__posting-description')
                inmueble['Título'] = title_container.text.strip()
                title_element = title_container.find_element(By.TAG_NAME, 'a')
                inmueble['Enlace'] = title_element.get_attribute('href')
            except Exception:
                inmueble['Título'] = 'No disponible'
                inmueble['Enlace'] = 'No disponible'

            try:
                price_element = listing.find_element(By.CSS_SELECTOR, 'div[data-qa="POSTING_CARD_PRICE"]')
                inmueble['Precio'] = price_element.text.strip()
            except Exception:
                inmueble['Precio'] = 'No disponible'

            try:
                address_element = listing.find_element(By.CSS_SELECTOR, '.postingLocations-module__location-block')
                inmueble['Dirección'] = address_element.text.strip()
            except Exception:
                inmueble['Dirección'] = 'No disponible'

            try:
                features = listing.find_elements(By.CSS_SELECTOR, 'span.postingMainFeatures-module__posting-main-features-listing')
                features_list = [feature.text.strip() for feature in features]
                inmueble['Características'] = ', '.join(features_list)
            except Exception:
                inmueble['Características'] = 'No disponible'
            
            inmueble['Colonia de Origen'] = nombre_colonia
            
            inmuebles_en_pagina.append(inmueble)
    except Exception as e:
        print(f"Error al extraer datos de la página: {e}")
    
    return inmuebles_en_pagina

## script principal

In [7]:
# 3. Leer la lista de colonias del archivo CSV
print("Leyendo la lista de colonias desde el archivo...")
lista_colonias = leer_colonias_csv(csv_colonias_filename)

if not lista_colonias:
    print("No se pudo obtener la lista de colonias. Terminando el script.")
    exit()

print(f"Se encontraron {len(lista_colonias)} colonias únicas para buscar.")

Leyendo la lista de colonias desde el archivo...
Se encontraron 1381 colonias únicas para buscar.


In [8]:
# 4. Configurar el WebDriver de Selenium
try:
    print("\nIniciando el navegador web sin detección...")
    driver = uc.Chrome(options=uc.ChromeOptions(), timeout=240)
    driver.maximize_window()
except WebDriverException as e:
    print(f"Error al iniciar el navegador: {e}")
    print("Asegúrate de tener la librería 'undetected-chromedriver' instalada.")
    exit()


Iniciando el navegador web sin detección...


In [9]:
try:
    # Asegurar que el directorio de salida exista
    os.makedirs(output_directory, exist_ok=True)
    
    # 5. Iterar a través de cada colonia y realizar el scraping
    for index, nombre_colonia in enumerate(lista_colonias):
        normalized_name = normalizar_nombre(nombre_colonia)
        
        colonia_filename = os.path.join(output_directory, f"{normalized_name}.csv")
        if os.path.exists(colonia_filename):
            print(f"Archivo para '{nombre_colonia}' ya existe. Saltando esta colonia. (van {index + 1} de {len(lista_colonias)}, faltan {len(lista_colonias) - (index + 1)})")
            continue
        
        print(f"\n--- Buscando en colonia: {nombre_colonia} (van {index + 1} de {len(lista_colonias)}, faltan {len(lista_colonias) - (index + 1)}) ---")

        # Lista para almacenar todos los datos de esta colonia
        inmuebles_por_colonia = []
        
        # URL inicial
        url_to_visit = f"{url_base}{normalized_name}.html"
        
        while True:
            print(f"Buscando en la URL: {url_to_visit}")

            try:
                driver.get(url_to_visit)
                wait = WebDriverWait(driver, 10)
                
                # Esperar a que el contenedor de resultados o el mensaje de no resultados aparezca
                wait.until(EC.any_of(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.postingsList-module__postings-container')),
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.emptyResults-module__empty-results'))
                ))

                # Verificar si la página tiene un mensaje de "no hay resultados"
                if len(driver.find_elements(By.CSS_SELECTOR, '.emptyResults-module__empty-results')) > 0:
                    print("La página no tiene resultados. Finalizando la paginación.")
                    break
                
                # Scrapeamos los datos de la página actual
                current_page_data = scrape_current_page(driver, nombre_colonia)
                
                if not current_page_data:
                    # Si el scraper no encuentra listings pero tampoco el mensaje de no resultados,
                    # asumimos que la paginación ha terminado.
                    print("No se encontraron propiedades en la página. Finalizando la paginación.")
                    break
                
                inmuebles_por_colonia.extend(current_page_data)
                
                print(f"Página scrapeada con éxito. Total de propiedades hasta ahora: {len(inmuebles_por_colonia)}")
                
                # Intentar encontrar el botón de "Siguiente" y obtener su href
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'a[data-qa="PAGING_NEXT"]')
                    # Actualizar la URL para la siguiente iteración
                    url_to_visit = next_button.get_attribute('href')
                    time.sleep(2) # Pausa para cargar la siguiente página
                except NoSuchElementException:
                    print("Botón 'Siguiente' no encontrado. Fin de la paginación.")
                    break # El botón no existe, hemos llegado a la última página
            
            except TimeoutException:
                print("Se superó el tiempo de espera. Probablemente no existen más páginas.")
                break
            except Exception as e:
                print(f"Ocurrió un error inesperado al procesar la URL '{url_to_visit}': {e}")
                break

            # --- FIN DE LA LÓGICA DE PAGINACIÓN ---

        if inmuebles_por_colonia:
            with open(colonia_filename, 'w', newline='', encoding='utf-8') as csvfile:
                fieldnames = ['Título', 'Enlace', 'Precio', 'Dirección', 'Características', 'Colonia de Origen']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(inmuebles_por_colonia)
            
            print(f"Datos de {len(inmuebles_por_colonia)} propiedades guardados en '{colonia_filename}'.")
        else:
            print(f"No se encontraron propiedades en ninguna página para la colonia '{nombre_colonia}'.")
            colonias_sin_resultados.append(nombre_colonia)
        
    
    # 9. Mostrar la lista de colonias que no tuvieron resultados
    if colonias_sin_resultados:
        print("\nLas siguientes colonias no arrojaron resultados:")
        for colonia in sorted(colonias_sin_resultados):
            print(f"- {colonia}")
    else:
        print("\n¡Todas las colonias procesadas tuvieron al menos una propiedad!")

except Exception as e:
    print(f"Ocurrió un error inesperado durante la ejecución principal: {e}")
finally:
    # 10. Cerrar el navegador al finalizar
    print("\nCerrando el navegador.")
    driver.quit()

print("\nProceso de scraping por colonias completado.")


Archivo para '10 de Abril' ya existe. Saltando esta colonia. (van 1 de 1381, faltan 1380)

--- Buscando en colonia: 10 de Mayo (van 2 de 1381, faltan 1379) ---
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-10-de-mayo.html
Se superó el tiempo de espera para la colonia '10 de Mayo'. No se encontraron resultados o la página no cargó correctamente.

--- Buscando en colonia: 12 de Diciembre (van 3 de 1381, faltan 1378) ---
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-12-de-diciembre.html
Se superó el tiempo de espera para la colonia '12 de Diciembre'. No se encontraron resultados o la página no cargó correctamente.

--- Buscando en colonia: 15 de Agosto (van 4 de 1381, faltan 1377) ---
Buscando en la URL: https://www.inmuebles24.com/departamentos-en-renta-en-15-de-agosto.html
Se superó el tiempo de espera para la colonia '15 de Agosto'. No se encontraron resultados o la página no cargó correctamente.
Archivo para '16 de Septiembre' ya